### Import lib

In [1]:
%load_ext lab_black

In [2]:
import os

os.chdir("/cerea_raid/users/dumontj/dev/coco2/dl")
import numpy as np
import sys
import pandas as pd
import xarray as xr
import tensorflow as tf
from sklearn import preprocessing
import matplotlib.pyplot as plt
import seaborn as sns

import matplotlib_functions

matplotlib_functions.setMatplotlibParam()
plt.viridis()
import joblib
import pickle
from include.generators import Generator
from saver import Saver
from include.callbacks import get_modelcheckpoint, ExtraValidation

import model_eval
from Data import Data_train, Data_eval
import wandb
from wandb.keras import WandbCallback, WandbMetricsLogger
import models.reg as rm
from include.loss import rmae
from include.loss import rmae

2023-02-24 10:46:23.325880: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


<Figure size 1920x1440 with 0 Axes>

In [3]:
generator = tf.keras.preprocessing.image.ImageDataGenerator(
    rotation_range=180,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    shear_range=45,
    zoom_range=0.2,
)

In [4]:
general_save_dir = "/cerea_raid/users/dumontj/dev/coco2/dl/nb_train/sim/"
project_name = "exp_inputs_lip"

### Data download

In [5]:
path_train_nc = "/libre/dumontj/coco2/dl-input/2km_forLip/train_dataset.nc"
path_valid_nc = "/libre/dumontj/coco2/dl-input/2km_forLip/valid_dataset.nc"
path_extra_valid_nc = "/libre/dumontj/coco2/dl-input/2km_Lip/valid_dataset.nc"

save_dir = os.path.join(general_save_dir, "lip")

# PLUME

In [10]:
data = Data_train(path_train_nc, path_valid_nc, path_extra_valid_nc)
data.prepare_input("plume")
data.prepare_output_inversion()

2023-02-23 16:01:54.422392: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-23 16:01:54.953877: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 14486 MB memory:  -> device: 0, name: Quadro RTX 5000, pci bus id: 0000:65:00.0, compute capability: 7.5


data.x.train.shape (34584, 64, 64, 1)


## EfficientNetB0 - no gen

In [7]:
exp_name = "plume_Eff_nogen"
wandb.init(project=project_name, name=exp_name)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: joffreydumont (cerea-daml). Use `wandb login --relogin` to force relogin


In [8]:
cbs = get_modelcheckpoint(True, [], os.path.join(save_dir, exp_name, "w_best.h5"))
history = ExtraValidation((data.x.extra_valid, data.y.extra_valid))
cbs.append(WandbMetricsLogger())
cbs.append(history)

In [9]:
model_builder = rm.Keras_reg_model_builder("EfficientNetB0", data.x.train.shape[1:], 1)
model = model_builder.get_model()
model.compile("adam", loss=tf.keras.losses.MeanAbsoluteError())

2023-02-10 19:04:48.066784: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-10 19:04:48.560631: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 14757 MB memory:  -> device: 0, name: Quadro RTX 5000, pci bus id: 0000:65:00.0, compute capability: 7.5


### Train 

In [10]:
model.fit(
    data.x.train,
    data.y.train,
    epochs=500,
    callbacks=cbs,
    validation_data=(data.x.valid, data.y.valid),
    verbose=1,
)

Epoch 1/500


2023-02-10 19:04:59.008873: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inmodel/efficientnetb0/block2b_drop/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer
2023-02-10 19:05:10.921267: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8401
2023-02-10 19:05:30.137419: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-02-10 19:05:33.232745: I tensorflow/compiler/xla/service/service.cc:173] XLA service 0x7fd7d1a1a1b0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-02-10 19:05:33.232804: I tensorflow/compiler/xla/service/service.cc:181]   StreamExecutor device (0): Quadro RTX 5000, Compute Capability 7.5
2023-02-10 19:05:34.208731: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash repr

393/393 [==============================] - ETA: 0s - loss: 3.0303
Epoch 1: val_loss improved from inf to 3.74985, saving model to /cerea_raid/users/dumontj/dev/coco2/dl/nb_train/sim/lip/plume_Eff_nogen/w_best.h5
extra_val_loss: 8.731101036071777
393/393 [==============================] - 85s 48ms/step - loss: 3.0303 - val_loss: 3.7498
Epoch 2/500
393/393 [==============================] - ETA: 0s - loss: 2.3026
Epoch 2: val_loss improved from 3.74985 to 2.25845, saving model to /cerea_raid/users/dumontj/dev/coco2/dl/nb_train/sim/lip/plume_Eff_nogen/w_best.h5
extra_val_loss: 7.639684200286865
393/393 [==============================] - 16s 42ms/step - loss: 2.3026 - val_loss: 2.2585
Epoch 3/500
393/393 [==============================] - ETA: 0s - loss: 2.1663
Epoch 3: val_loss did not improve from 2.25845
extra_val_loss: 9.045520782470703
393/393 [==============================] - 16s 41ms/step - loss: 2.1663 - val_loss: 2.2705
Epoch 4/500
393/393 [==============================] - ETA: 

### save

In [12]:
model.save(os.path.join(save_dir, exp_name, "w_last.h5"))
joblib.dump(data.x.scaler, os.path.join(save_dir, exp_name, "scaler.save"))

['/cerea_raid/users/dumontj/dev/coco2/dl/nb_train/sim/lip/plume_Eff_nogen/scaler.save']

## EfficientNetB0 - with gen

In [13]:
exp_name = "plume_Eff_gen"
wandb.init(project=project_name, name=exp_name)

epoch/epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
epoch/learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/loss,█▆▄▃▃▃▂▂▂▂▂▂▂▂▂▂▂▁▂▁▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/val_loss,█▂▂▂▂▁▂▁▂▁▂▂▁▂▂▃▂▂▂▂▂▄▂▂▂▂▂▂▂▂▂▂▃▃▂▂▃▃▂▃
extra_val_loss,█▄▁▃▃▄▃▃▃▃▃▄▃▄▃▃▃▃▄▄▃▅▃▃▄▄▄▄▃▄▃▃▄▃▃▃▅▃▃▄
epoch/epoch,499
epoch/learning_rate,0.001
epoch/loss,0.29915
epoch/val_loss,1.74727
extra_val_loss,7.65965


In [14]:
cbs = get_modelcheckpoint(True, [], os.path.join(save_dir, exp_name, "w_best.h5"))
history = ExtraValidation((data.x.extra_valid, data.y.extra_valid))
cbs.append(WandbMetricsLogger())
cbs.append(history)

In [15]:
model_builder = rm.Keras_reg_model_builder("EfficientNetB0", data.x.train.shape[1:], 1)
model = model_builder.get_model()
model.compile("adam", loss=tf.keras.losses.MeanAbsoluteError())

### Train 

In [16]:
model.fit(
    generator.flow(data.x.train, data.y.train),
    epochs=1000,
    callbacks=cbs,
    validation_data=(data.x.valid, data.y.valid),
    verbose=1,
)

Epoch 1/1000


2023-02-13 10:31:14.534740: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inmodel_1/efficientnetb0/block2b_drop/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


392/393 [============================>.] - ETA: 0s - loss: 3.7182
Epoch 1: val_loss improved from inf to 3.81540, saving model to /cerea_raid/users/dumontj/dev/coco2/dl/nb_train/sim/lip/plume_Eff_gen/w_best.h5
extra_val_loss: 8.791255950927734
393/393 [==============================] - 44s 54ms/step - loss: 3.7194 - val_loss: 3.8154
Epoch 2/1000
393/393 [==============================] - ETA: 0s - loss: 2.8097
Epoch 2: val_loss did not improve from 3.81540
extra_val_loss: 11.358316421508789
393/393 [==============================] - 20s 50ms/step - loss: 2.8097 - val_loss: 8.8648
Epoch 3/1000
392/393 [============================>.] - ETA: 0s - loss: 2.6263
Epoch 3: val_loss improved from 3.81540 to 2.67680, saving model to /cerea_raid/users/dumontj/dev/coco2/dl/nb_train/sim/lip/plume_Eff_gen/w_best.h5
extra_val_loss: 7.605177879333496
393/393 [==============================] - 21s 53ms/step - loss: 2.6257 - val_loss: 2.6768
Epoch 4/1000
392/393 [============================>.] - ETA: 

### save

In [17]:
model.save(os.path.join(save_dir, exp_name, "w_last.h5"))
joblib.dump(data.x.scaler, os.path.join(save_dir, exp_name, "scaler.save"))

['/cerea_raid/users/dumontj/dev/coco2/dl/nb_train/sim/lip/plume_Eff_gen/scaler.save']

## Linear baseline

In [18]:
exp_name = "plume_linear"
wandb.init(project=project_name, name=exp_name)

epoch/epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
epoch/learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/loss,█▆▅▅▄▄▄▃▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▁▂▁▂▁▁▁▁▁▁▁▁
epoch/val_loss,▇█▄▂▃▂▃▂▃▂▂▃▃▃▁▁▃▃▂▃▂▂▁▁▁▂▂▂▃▄▁▂▆▁▄▅▆▂▅▂
extra_val_loss,▆▁█▇▆▅▅▆▄▃▅▇▅▅▅▅▅▅▆▅▅▃▆▄▄▅▅▆▄▃▅▄▃▅▄▃▄▅▃▅
epoch/epoch,999
epoch/learning_rate,0.001
epoch/loss,1.24627
epoch/val_loss,1.80809
extra_val_loss,7.54334


In [19]:
cbs = get_modelcheckpoint(True, [], os.path.join(save_dir, exp_name, "w_best.h5"))
history = ExtraValidation((data.x.extra_valid, data.y.extra_valid))
cbs.append(WandbMetricsLogger())
cbs.append(history)

In [20]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Conv2D(16, (3, 3), input_shape=data.x.fields_input_shape))
model.add(tf.keras.layers.MaxPooling2D((2, 2)))
model.add(tf.keras.layers.Conv2D(16, (3, 3)))
model.add(tf.keras.layers.MaxPooling2D((2, 2)))
model.add(tf.keras.layers.Conv2D(16, (3, 3)))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(16))
model.add(tf.keras.layers.Dense(1))

model.compile("adam", loss="MeanAbsoluteError")

### Train 

In [21]:
model.fit(
    data.x.train,
    data.y.train,
    epochs=200,
    callbacks=cbs,
    validation_data=(data.x.valid, data.y.valid),
    verbose=1,
)

Epoch 1/200
393/393 [==============================] - ETA: 0s - loss: 3.1041
Epoch 1: val_loss improved from inf to 2.82344, saving model to /cerea_raid/users/dumontj/dev/coco2/dl/nb_train/sim/lip/plume_linear/w_best.h5
extra_val_loss: 7.381600379943848
393/393 [==============================] - 5s 7ms/step - loss: 3.1041 - val_loss: 2.8234
Epoch 2/200
390/393 [============================>.] - ETA: 0s - loss: 2.7126
Epoch 2: val_loss improved from 2.82344 to 2.76234, saving model to /cerea_raid/users/dumontj/dev/coco2/dl/nb_train/sim/lip/plume_linear/w_best.h5
extra_val_loss: 7.354020118713379
393/393 [==============================] - 3s 7ms/step - loss: 2.7135 - val_loss: 2.7623
Epoch 3/200
390/393 [============================>.] - ETA: 0s - loss: 2.6484
Epoch 3: val_loss improved from 2.76234 to 2.71337, saving model to /cerea_raid/users/dumontj/dev/coco2/dl/nb_train/sim/lip/plume_linear/w_best.h5
extra_val_loss: 7.773451805114746
393/393 [==============================] - 3s 7ms

### save

In [22]:
model.save(os.path.join(save_dir, exp_name, "w_last.h5"))
joblib.dump(data.x.scaler, os.path.join(save_dir, exp_name, "scaler.save"))

['/cerea_raid/users/dumontj/dev/coco2/dl/nb_train/sim/lip/plume_linear/scaler.save']

## Research of a new model

In [11]:
exp_name = "plume_modelX"
wandb.init(project=project_name, name=exp_name)

In [12]:
cbs = get_modelcheckpoint(True, [], os.path.join(save_dir, exp_name, "w_best.h5"))
history = ExtraValidation((data.x.extra_valid, data.y.extra_valid))
cbs.append(WandbMetricsLogger())
cbs.append(history)

In [13]:
inputs = tf.keras.layers.Input((64, 64, 1))

x = tf.keras.layers.Conv2D(32, kernel_size=(3, 3), activation="elu", strides=1)(inputs)
x = tf.keras.layers.Dropout(0.1)(x)
x = tf.keras.layers.Conv2D(32, kernel_size=(3, 3), activation="elu", strides=1)(x)
x = tf.keras.layers.MaxPooling2D(pool_size=(2, 2), padding="valid", strides=2)(x)
x = tf.keras.layers.BatchNormalization()(x)

x = tf.keras.layers.Conv2D(32, kernel_size=(3, 3), activation="elu", strides=1)(x)
x = tf.keras.layers.Dropout(0.2)(x)
x = tf.keras.layers.Conv2D(64, kernel_size=(3, 3), activation="elu", strides=1)(x)
x = tf.keras.layers.BatchNormalization()(x)

x = tf.keras.layers.Conv2D(64, kernel_size=(3, 3), activation="elu", strides=1)(x)
x = tf.keras.layers.Dropout(0.2)(x)
x = tf.keras.layers.Conv2D(64, kernel_size=(3, 3), activation="elu", strides=1)(x)
x = tf.keras.layers.MaxPooling2D(pool_size=(2, 2), padding="valid", strides=2)(x)

x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(1)(x)
outputs = tf.keras.layers.LeakyReLU(alpha=0.3)(x)

model = tf.keras.Model([inputs], outputs)

model.compile("adam", loss="MeanAbsoluteError")

### Train 

In [14]:
model.fit(
    data.x.train,
    data.y.train,
    epochs=500,
    callbacks=cbs,
    validation_data=(data.x.valid, data.y.valid),
    verbose=1,
)

Epoch 1/500


2023-02-23 16:02:08.537299: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inmodel/dropout/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer
2023-02-23 16:02:09.369115: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8401
2023-02-23 16:02:10.782617: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-02-23 16:02:13.150847: I tensorflow/compiler/xla/service/service.cc:173] XLA service 0x7fdcf73c0190 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-02-23 16:02:13.150901: I tensorflow/compiler/xla/service/service.cc:181]   StreamExecutor device (0): Quadro RTX 5000, Compute Capability 7.5
2023-02-23 16:02:13.159779: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var 

1081/1081 [==============================] - ETA: 0s - loss: 5.3185
Epoch 1: val_loss improved from inf to 6.87956, saving model to /cerea_raid/users/dumontj/dev/coco2/dl/nb_train/sim/lip/plume_modelX/w_best.h5
extra_val_loss: 7.19863224029541
1081/1081 [==============================] - 26s 13ms/step - loss: 5.3185 - val_loss: 6.8796
Epoch 2/500
1081/1081 [==============================] - ETA: 0s - loss: 5.0476
Epoch 2: val_loss improved from 6.87956 to 5.12422, saving model to /cerea_raid/users/dumontj/dev/coco2/dl/nb_train/sim/lip/plume_modelX/w_best.h5
extra_val_loss: 5.630082130432129
1081/1081 [==============================] - 13s 12ms/step - loss: 5.0476 - val_loss: 5.1242
Epoch 3/500
1081/1081 [==============================] - ETA: 0s - loss: 4.9013
Epoch 3: val_loss improved from 5.12422 to 5.04713, saving model to /cerea_raid/users/dumontj/dev/coco2/dl/nb_train/sim/lip/plume_modelX/w_best.h5
extra_val_loss: 5.607795238494873
1081/1081 [==============================] - 14s

Exception ignored in: <function WeakSet.__init__.<locals>._remove at 0x7fe439f28ca0>
Traceback (most recent call last):
  File "/profils_cerea/dumontj/mambaforge/envs/gpu/lib/python3.9/_weakrefset.py", line 39, in _remove
    def _remove(item, selfref=ref(self)):
KeyboardInterrupt: 


KeyboardInterrupt: 

### save

In [27]:
model.save(os.path.join(save_dir, exp_name, "w_last.h5"))
joblib.dump(data.x.scaler, os.path.join(save_dir, exp_name, "scaler.save"))

['/cerea_raid/users/dumontj/dev/coco2/dl/nb_train/sim/lip/plume_modelX/scaler.save']

# PLUME + PLUME PREC

In [28]:
wandb.finish()

epoch/epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
epoch/learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/loss,█▅▅▄▃▃▃▂▂▂▂▂▂▂▂▁▂▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/val_loss,█▃▂▃▂▂▂▂▂▃▂▁▂▃▃▂▄▂▂▂▃▂▂▂▂▃▁▂▂▃▁▂▂▁▂▂▂▂▂▂
extra_val_loss,▇▆▅▃▄▄▃▄▂▂▄▅▄▄▅▆█▄▅▃▂▅▆▂▄▁▄▃█▇▃▄▄▄▃▄▅▄▃▄
epoch/epoch,499
epoch/learning_rate,0.001
epoch/loss,0.83311
epoch/val_loss,1.69807
extra_val_loss,7.60919


In [29]:
data = Data_train(path_train_nc, path_valid_nc, path_extra_valid_nc)
data.prepare_input("plume", "plume_prec")
data.prepare_output_inversion()

data.x.train.shape (12576, 64, 64, 2)


## EfficientNetB0 - no gen

In [30]:
exp_name = "plume_plumeprec_Eff_nogen"
wandb.init(project=project_name, name=exp_name)

In [31]:
cbs = get_modelcheckpoint(True, [], os.path.join(save_dir, exp_name, "w_best.h5"))
history = ExtraValidation((data.x.extra_valid, data.y.extra_valid))
cbs.append(WandbMetricsLogger())
cbs.append(history)

In [32]:
model_builder = rm.Keras_reg_model_builder("EfficientNetB0", data.x.train.shape[1:], 1)
model = model_builder.get_model()
model.compile("adam", loss=tf.keras.losses.MeanAbsoluteError())

/profils_cerea/dumontj/mambaforge/envs/gpu/lib/python3.9/site-packages/keras/applications/efficientnet.py:321: UserWarning: This model usually expects 1 or 3 input channels. However, it was passed an input_shape with 2 input channels.
  input_shape = imagenet_utils.obtain_input_shape(


### Train 

In [33]:
model.fit(
    data.x.train,
    data.y.train,
    epochs=500,
    callbacks=cbs,
    validation_data=(data.x.valid, data.y.valid),
    verbose=1,
)

Epoch 1/500


2023-02-13 16:48:37.308698: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inmodel_3/efficientnetb0/block2b_drop/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


393/393 [==============================] - ETA: 0s - loss: 3.1715
Epoch 1: val_loss improved from inf to 3.93071, saving model to /cerea_raid/users/dumontj/dev/coco2/dl/nb_train/sim/lip/plume_plumeprec_Eff_nogen/w_best.h5
extra_val_loss: 9.057893753051758
393/393 [==============================] - 39s 44ms/step - loss: 3.1715 - val_loss: 3.9307
Epoch 2/500
393/393 [==============================] - ETA: 0s - loss: 2.2084
Epoch 2: val_loss improved from 3.93071 to 2.83419, saving model to /cerea_raid/users/dumontj/dev/coco2/dl/nb_train/sim/lip/plume_plumeprec_Eff_nogen/w_best.h5
extra_val_loss: 6.90962028503418
393/393 [==============================] - 17s 43ms/step - loss: 2.2084 - val_loss: 2.8342
Epoch 3/500
393/393 [==============================] - ETA: 0s - loss: 1.9999
Epoch 3: val_loss did not improve from 2.83419
extra_val_loss: 7.065821170806885
393/393 [==============================] - 16s 41ms/step - loss: 1.9999 - val_loss: 2.8662
Epoch 4/500
393/393 [====================

### save

In [34]:
model.save(os.path.join(save_dir, exp_name, "w_last.h5"))
joblib.dump(data.x.scaler, os.path.join(save_dir, exp_name, "scaler.save"))

['/cerea_raid/users/dumontj/dev/coco2/dl/nb_train/sim/lip/plume_plumeprec_Eff_nogen/scaler.save']

## EfficientNetB0 - with gen

In [35]:
exp_name = "plume_plumeprec_Eff_gen"
wandb.init(project=project_name, name=exp_name)

epoch/epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
epoch/learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/loss,█▆▄▄▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▁▂▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/val_loss,█▃█▂▁▁▁▂▁▂▁▃▂▂▂▂▂▂▂▂▁▂▁▁▂▂▂▂▁▂▁▂▂▂▁▁▂▂▂▂
extra_val_loss,▅▁▇▇▃▆▇█▅▆▄▄▁▁▅▅▄▄▄▆▅▅▅▅▄▇▄▄▄▅▅▅▃▃▂▃▃▃▃▃
epoch/epoch,499
epoch/learning_rate,0.001
epoch/loss,0.14217
epoch/val_loss,1.58779
extra_val_loss,7.39063


In [38]:
cbs = get_modelcheckpoint(True, [], os.path.join(save_dir, exp_name, "w_best.h5"))
history = ExtraValidation((data.x.extra_valid, data.y.extra_valid))
cbs.append(WandbMetricsLogger())
cbs.append(history)

In [39]:
model_builder = rm.Keras_reg_model_builder("EfficientNetB0", data.x.train.shape[1:], 1)
model = model_builder.get_model()
model.compile("adam", loss=tf.keras.losses.MeanAbsoluteError())

/profils_cerea/dumontj/mambaforge/envs/gpu/lib/python3.9/site-packages/keras/applications/efficientnet.py:321: UserWarning: This model usually expects 1 or 3 input channels. However, it was passed an input_shape with 2 input channels.
  input_shape = imagenet_utils.obtain_input_shape(


### Train 

In [40]:
model.fit(
    generator.flow(data.x.train, data.y.train),
    epochs=1000,
    callbacks=cbs,
    validation_data=(data.x.valid, data.y.valid),
    verbose=1,
)

Epoch 1/1000


/profils_cerea/dumontj/mambaforge/envs/gpu/lib/python3.9/site-packages/keras/preprocessing/image.py:766: UserWarning: NumpyArrayIterator is set to use the data format convention "channels_last" (channels on axis 3), i.e. expected either 1, 3, or 4 channels on axis 3. However, it was passed an array with shape (12576, 64, 64, 2) (2 channels).
  warnings.warn(
2023-02-13 19:12:01.204274: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inmodel_4/efficientnetb0/block2b_drop/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


393/393 [==============================] - ETA: 0s - loss: 3.8216
Epoch 1: val_loss improved from inf to 3.64880, saving model to /cerea_raid/users/dumontj/dev/coco2/dl/nb_train/sim/lip/plume_plumeprec_Eff_gen/w_best.h5
extra_val_loss: 8.248172760009766
393/393 [==============================] - 44s 54ms/step - loss: 3.8216 - val_loss: 3.6488
Epoch 2/1000
393/393 [==============================] - ETA: 0s - loss: 2.7438
Epoch 2: val_loss improved from 3.64880 to 3.18247, saving model to /cerea_raid/users/dumontj/dev/coco2/dl/nb_train/sim/lip/plume_plumeprec_Eff_gen/w_best.h5
extra_val_loss: 6.936817646026611
393/393 [==============================] - 20s 51ms/step - loss: 2.7438 - val_loss: 3.1825
Epoch 3/1000
393/393 [==============================] - ETA: 0s - loss: 2.5172
Epoch 3: val_loss improved from 3.18247 to 2.15455, saving model to /cerea_raid/users/dumontj/dev/coco2/dl/nb_train/sim/lip/plume_plumeprec_Eff_gen/w_best.h5
extra_val_loss: 7.501738548278809
393/393 [=============

### save

In [41]:
model.save(os.path.join(save_dir, exp_name, "w_last.h5"))
joblib.dump(data.x.scaler, os.path.join(save_dir, exp_name, "scaler.save"))

['/cerea_raid/users/dumontj/dev/coco2/dl/nb_train/sim/lip/plume_plumeprec_Eff_gen/scaler.save']

## Linear baseline

In [42]:
exp_name = "plume_plumeprec_linear"
wandb.init(project=project_name, name=exp_name)

epoch/epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
epoch/learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/loss,█▆▅▅▅▄▄▄▄▃▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▁▂▁▁▁▁▁▁▁▁
epoch/val_loss,█▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
extra_val_loss,▁█▇▆▅▅▅▅▆▅▆▅▅▄▅▆▅▆▅▅▆▅▄▅▅▄▄▄▃▅▄▅▄▄▅▄▅▄▅▆
epoch/epoch,999
epoch/learning_rate,0.001
epoch/loss,1.02365
epoch/val_loss,1.52949
extra_val_loss,7.3595


In [43]:
cbs = get_modelcheckpoint(True, [], os.path.join(save_dir, exp_name, "w_best.h5"))
history = ExtraValidation((data.x.extra_valid, data.y.extra_valid))
cbs.append(WandbMetricsLogger())
cbs.append(history)

In [44]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Conv2D(16, (3, 3), input_shape=data.x.fields_input_shape))
model.add(tf.keras.layers.MaxPooling2D((2, 2)))
model.add(tf.keras.layers.Conv2D(16, (3, 3)))
model.add(tf.keras.layers.MaxPooling2D((2, 2)))
model.add(tf.keras.layers.Conv2D(16, (3, 3)))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(16))
model.add(tf.keras.layers.Dense(1))

model.compile("adam", loss="MeanAbsoluteError")

### Train 

In [45]:
model.fit(
    data.x.train,
    data.y.train,
    epochs=200,
    callbacks=cbs,
    validation_data=(data.x.valid, data.y.valid),
    verbose=1,
)

Epoch 1/200
387/393 [============================>.] - ETA: 0s - loss: 3.1333
Epoch 1: val_loss improved from inf to 2.78929, saving model to /cerea_raid/users/dumontj/dev/coco2/dl/nb_train/sim/lip/plume_plumeprec_linear/w_best.h5
extra_val_loss: 6.792510986328125
393/393 [==============================] - 4s 8ms/step - loss: 3.1271 - val_loss: 2.7893
Epoch 2/200
391/393 [============================>.] - ETA: 0s - loss: 2.6722
Epoch 2: val_loss improved from 2.78929 to 2.64715, saving model to /cerea_raid/users/dumontj/dev/coco2/dl/nb_train/sim/lip/plume_plumeprec_linear/w_best.h5
extra_val_loss: 6.763813495635986
393/393 [==============================] - 3s 7ms/step - loss: 2.6724 - val_loss: 2.6472
Epoch 3/200
392/393 [============================>.] - ETA: 0s - loss: 2.5268
Epoch 3: val_loss improved from 2.64715 to 2.64337, saving model to /cerea_raid/users/dumontj/dev/coco2/dl/nb_train/sim/lip/plume_plumeprec_linear/w_best.h5
extra_val_loss: 7.982030868530273
393/393 [==========

### save

In [46]:
model.save(os.path.join(save_dir, exp_name, "w_last.h5"))
joblib.dump(data.x.scaler, os.path.join(save_dir, exp_name, "scaler.save"))

['/cerea_raid/users/dumontj/dev/coco2/dl/nb_train/sim/lip/plume_plumeprec_linear/scaler.save']

## Research of a new model

In [47]:
exp_name = "plume_plumeprec_modelX"
wandb.init(project=project_name, name=exp_name)

epoch/epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
epoch/learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/loss,█▆▅▅▄▄▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/val_loss,█▆▃▂▂▁▁▁▁▂▁▁▂▁▂▁▂▂▁▂▃▂▂▂▄▂▂▃▂▂▂▃▂▃▄▃▃▄▄▄
extra_val_loss,▃█▁▇▃▇▅▇▆▃▅▄▇▅▅▆▇▇▅▇▅▆▇▆▄▆▆▇▅▇▆▇▇▆▄▆▆▆█▆
epoch/epoch,199
epoch/learning_rate,0.001
epoch/loss,1.2936
epoch/val_loss,2.42119
extra_val_loss,7.69349


In [48]:
cbs = get_modelcheckpoint(True, [], os.path.join(save_dir, exp_name, "w_best.h5"))
history = ExtraValidation((data.x.extra_valid, data.y.extra_valid))
cbs.append(WandbMetricsLogger())
cbs.append(history)

In [51]:
inputs = tf.keras.layers.Input((64, 64, 2))

x = tf.keras.layers.Conv2D(32, kernel_size=(3, 3), activation="elu", strides=1)(inputs)
x = tf.keras.layers.Dropout(0.1)(x)
x = tf.keras.layers.Conv2D(32, kernel_size=(3, 3), activation="elu", strides=1)(x)
x = tf.keras.layers.MaxPooling2D(pool_size=(2, 2), padding="valid", strides=2)(x)
x = tf.keras.layers.BatchNormalization()(x)

x = tf.keras.layers.Conv2D(32, kernel_size=(3, 3), activation="elu", strides=1)(x)
x = tf.keras.layers.Dropout(0.2)(x)
x = tf.keras.layers.Conv2D(64, kernel_size=(3, 3), activation="elu", strides=1)(x)
x = tf.keras.layers.BatchNormalization()(x)

x = tf.keras.layers.Conv2D(64, kernel_size=(3, 3), activation="elu", strides=1)(x)
x = tf.keras.layers.Dropout(0.2)(x)
x = tf.keras.layers.Conv2D(64, kernel_size=(3, 3), activation="elu", strides=1)(x)
x = tf.keras.layers.MaxPooling2D(pool_size=(2, 2), padding="valid", strides=2)(x)

x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(1)(x)
outputs = tf.keras.layers.LeakyReLU(alpha=0.3)(x)

model = tf.keras.Model([inputs], outputs)

model.compile("adam", loss="MeanAbsoluteError")

### Train 

In [52]:
model.fit(
    data.x.train,
    data.y.train,
    epochs=500,
    callbacks=cbs,
    validation_data=(data.x.valid, data.y.valid),
    verbose=1,
)

Epoch 1/500


2023-02-14 10:05:05.497800: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inmodel_6/dropout_6/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


392/393 [============================>.] - ETA: 0s - loss: 2.6424
Epoch 1: val_loss improved from inf to 2.69670, saving model to /cerea_raid/users/dumontj/dev/coco2/dl/nb_train/sim/lip/plume_plumeprec_modelX/w_best.h5
extra_val_loss: 8.011743545532227
393/393 [==============================] - 7s 13ms/step - loss: 2.6419 - val_loss: 2.6967
Epoch 2/500
393/393 [==============================] - ETA: 0s - loss: 2.2314
Epoch 2: val_loss did not improve from 2.69670
extra_val_loss: 9.86306381225586
393/393 [==============================] - 5s 12ms/step - loss: 2.2314 - val_loss: 2.8764
Epoch 3/500
391/393 [============================>.] - ETA: 0s - loss: 2.1515
Epoch 3: val_loss improved from 2.69670 to 2.08151, saving model to /cerea_raid/users/dumontj/dev/coco2/dl/nb_train/sim/lip/plume_plumeprec_modelX/w_best.h5
extra_val_loss: 8.431042671203613
393/393 [==============================] - 5s 12ms/step - loss: 2.1509 - val_loss: 2.0815
Epoch 4/500
392/393 [============================>

### save

In [53]:
model.save(os.path.join(save_dir, exp_name, "w_last.h5"))
joblib.dump(data.x.scaler, os.path.join(save_dir, exp_name, "scaler.save"))

['/cerea_raid/users/dumontj/dev/coco2/dl/nb_train/sim/lip/plume_plumeprec_modelX/scaler.save']

# PLUME + PLUME PREC + WINDS

In [54]:
data = Data_train(path_train_nc, path_valid_nc, path_extra_valid_nc)
data.prepare_input("plume", "plume_prec", "u_wind", "v_wind")
data.prepare_output_inversion()

data.x.train.shape (12576, 64, 64, 4)


## EfficientNetB0 - no gen

In [55]:
exp_name = "plume_plumeprec_winds_Eff_nogen"
wandb.init(project=project_name, name=exp_name)

epoch/epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
epoch/learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/loss,█▅▄▄▃▃▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/val_loss,█▆▅▆▃▂▂▂▄▂▃▂▂▂▃▁▂▁▁▂▂▂▂▂▂▁▁▃▂▁▂▂▁▂▂▂▂▂▁▁
extra_val_loss,▄▅▆▃▂▂▂▄█▅▄▁▃▄▅▃▄▂▂▃▅▃▃▃▇▃▅▁▄▃▂▄▃▁▅▅▃▂▃▄
epoch/epoch,499
epoch/learning_rate,0.001
epoch/loss,0.73083
epoch/val_loss,1.64568
extra_val_loss,6.25778


In [56]:
cbs = get_modelcheckpoint(True, [], os.path.join(save_dir, exp_name, "w_best.h5"))
history = ExtraValidation((data.x.extra_valid, data.y.extra_valid))
cbs.append(WandbMetricsLogger())
cbs.append(history)

In [57]:
model_builder = rm.Keras_reg_model_builder("EfficientNetB0", data.x.train.shape[1:], 1)
model = model_builder.get_model()
model.compile("adam", loss=tf.keras.losses.MeanAbsoluteError())

/profils_cerea/dumontj/mambaforge/envs/gpu/lib/python3.9/site-packages/keras/applications/efficientnet.py:321: UserWarning: This model usually expects 1 or 3 input channels. However, it was passed an input_shape with 4 input channels.
  input_shape = imagenet_utils.obtain_input_shape(


### Train 

In [58]:
model.fit(
    data.x.train,
    data.y.train,
    epochs=500,
    callbacks=cbs,
    validation_data=(data.x.valid, data.y.valid),
    verbose=1,
)

Epoch 1/500


2023-02-14 10:45:20.223867: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inmodel_7/efficientnetb0/block2b_drop/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


393/393 [==============================] - ETA: 0s - loss: 2.9239
Epoch 1: val_loss improved from inf to 3.93539, saving model to /cerea_raid/users/dumontj/dev/coco2/dl/nb_train/sim/lip/plume_plumeprec_winds_Eff_nogen/w_best.h5
extra_val_loss: 9.154890060424805
393/393 [==============================] - 40s 46ms/step - loss: 2.9239 - val_loss: 3.9354
Epoch 2/500
393/393 [==============================] - ETA: 0s - loss: 2.0345
Epoch 2: val_loss improved from 3.93539 to 1.94884, saving model to /cerea_raid/users/dumontj/dev/coco2/dl/nb_train/sim/lip/plume_plumeprec_winds_Eff_nogen/w_best.h5
extra_val_loss: 6.440787315368652
393/393 [==============================] - 17s 43ms/step - loss: 2.0345 - val_loss: 1.9488
Epoch 3/500
392/393 [============================>.] - ETA: 0s - loss: 1.8341
Epoch 3: val_loss did not improve from 1.94884
extra_val_loss: 7.975879192352295
393/393 [==============================] - 16s 41ms/step - loss: 1.8345 - val_loss: 2.2621
Epoch 4/500
393/393 [=======

### save

In [59]:
model.save(os.path.join(save_dir, exp_name, "w_last.h5"))
joblib.dump(data.x.scaler, os.path.join(save_dir, exp_name, "scaler.save"))

['/cerea_raid/users/dumontj/dev/coco2/dl/nb_train/sim/lip/plume_plumeprec_winds_Eff_nogen/scaler.save']

## EfficientNetB0 - with gen

In [60]:
exp_name = "plume_plumeprec_winds_Eff_gen"
wandb.init(project=project_name, name=exp_name)

epoch/epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
epoch/learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/loss,█▆▅▄▄▃▃▃▃▂▃▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/val_loss,█▇▁▂▂▂▁▁▁▃▆▃▂▁▁▁▁▂▁▂▂▁▂▂▁▁▄▂▂▂▁▂▂▂▂▂▂▂▂▂
extra_val_loss,▁█▃▂▃▃▃▄▃▄▃▃▄▄▃▃▄▃▄▄▄▃▄▄▄▃▃▄▅▄▃▄▄▄▄▄▄▄▄▃
epoch/epoch,499
epoch/learning_rate,0.001
epoch/loss,0.16792
epoch/val_loss,1.42667
extra_val_loss,7.23652


In [61]:
cbs = get_modelcheckpoint(True, [], os.path.join(save_dir, exp_name, "w_best.h5"))
history = ExtraValidation((data.x.extra_valid, data.y.extra_valid))
cbs.append(WandbMetricsLogger())
cbs.append(history)

In [62]:
model_builder = rm.Keras_reg_model_builder("EfficientNetB0", data.x.train.shape[1:], 1)
model = model_builder.get_model()
model.compile("adam", loss=tf.keras.losses.MeanAbsoluteError())

/profils_cerea/dumontj/mambaforge/envs/gpu/lib/python3.9/site-packages/keras/applications/efficientnet.py:321: UserWarning: This model usually expects 1 or 3 input channels. However, it was passed an input_shape with 4 input channels.
  input_shape = imagenet_utils.obtain_input_shape(


### Train 

In [ ]:
model.fit(
    generator.flow(data.x.train, data.y.train),
    epochs=1000,
    callbacks=cbs,
    validation_data=(data.x.valid, data.y.valid),
    verbose=1
)

Epoch 1/1000


2023-02-14 12:59:40.352784: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inmodel_8/efficientnetb0/block2b_drop/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


393/393 [==============================] - ETA: 0s - loss: 3.8807
Epoch 1: val_loss improved from inf to 4.14232, saving model to /cerea_raid/users/dumontj/dev/coco2/dl/nb_train/sim/lip/plume_plumeprec_winds_Eff_gen/w_best.h5
extra_val_loss: 9.421329498291016
393/393 [==============================] - 44s 57ms/step - loss: 3.8807 - val_loss: 4.1423
Epoch 2/1000
393/393 [==============================] - ETA: 0s - loss: 3.1564
Epoch 2: val_loss did not improve from 4.14232
extra_val_loss: 10.923891067504883
393/393 [==============================] - 20s 52ms/step - loss: 3.1564 - val_loss: 5.1872
Epoch 3/1000
392/393 [============================>.] - ETA: 0s - loss: 2.4476
Epoch 3: val_loss improved from 4.14232 to 2.88974, saving model to /cerea_raid/users/dumontj/dev/coco2/dl/nb_train/sim/lip/plume_plumeprec_winds_Eff_gen/w_best.h5
extra_val_loss: 5.87790584564209
393/393 [==============================] - 22s 55ms/step - loss: 2.4455 - val_loss: 2.8897
Epoch 4/1000
393/393 [========

### save

In [ ]:
model.save(os.path.join(save_dir, exp_name, "w_last.h5"))
joblib.dump(data.x.scaler, os.path.join(save_dir, exp_name, "scaler.save"))

## Linear baseline

In [ ]:
exp_name = "plume_plumeprec_winds_linear"
wandb.init(project=project_name, name=exp_name)

In [ ]:
cbs = get_modelcheckpoint(True, [], os.path.join(save_dir, exp_name, "w_best.h5"))
history = ExtraValidation((data.x.extra_valid, data.y.extra_valid))
cbs.append(WandbMetricsLogger())
cbs.append(history)

In [ ]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Conv2D(16, (3, 3), input_shape=data.x.fields_input_shape))
model.add(tf.keras.layers.MaxPooling2D((2, 2)))
model.add(tf.keras.layers.Conv2D(16, (3, 3)))
model.add(tf.keras.layers.MaxPooling2D((2, 2)))
model.add(tf.keras.layers.Conv2D(16, (3, 3)))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(16))
model.add(tf.keras.layers.Dense(1))

model.compile("adam", loss="MeanAbsoluteError")

### Train 

In [ ]:
model.fit(
    data.x.train,
    data.y.train,
    epochs=200,
    callbacks=cbs,
    validation_data=(data.x.valid, data.y.valid),
    verbose=1,
)

### save

In [ ]:
model.save(os.path.join(save_dir, exp_name, "w_last.h5"))
joblib.dump(data.x.scaler, os.path.join(save_dir, exp_name, "scaler.save"))

## Research of a new model

In [ ]:
exp_name = "plume_plumeprec_winds_modelX"
wandb.init(project=project_name, name=exp_name)

In [ ]:
cbs = get_modelcheckpoint(True, [], os.path.join(save_dir, exp_name, "w_best.h5"))
history = ExtraValidation((data.x.extra_valid, data.y.extra_valid))
cbs.append(WandbMetricsLogger())
cbs.append(history)

In [ ]:
inputs = tf.keras.layers.Input((64, 64, 4))

x = tf.keras.layers.Conv2D(32, kernel_size=(3, 3), activation="elu", strides=1)(inputs)
x = tf.keras.layers.Dropout(0.1)(x)
x = tf.keras.layers.Conv2D(32, kernel_size=(3, 3), activation="elu", strides=1)(x)
x = tf.keras.layers.MaxPooling2D(pool_size=(2, 2), padding="valid", strides=2)(x)
x = tf.keras.layers.BatchNormalization()(x)

x = tf.keras.layers.Conv2D(32, kernel_size=(3, 3), activation="elu", strides=1)(x)
x = tf.keras.layers.Dropout(0.2)(x)
x = tf.keras.layers.Conv2D(64, kernel_size=(3, 3), activation="elu", strides=1)(x)
x = tf.keras.layers.BatchNormalization()(x)

x = tf.keras.layers.Conv2D(64, kernel_size=(3, 3), activation="elu", strides=1)(x)
x = tf.keras.layers.Dropout(0.2)(x)
x = tf.keras.layers.Conv2D(64, kernel_size=(3, 3), activation="elu", strides=1)(x)
x = tf.keras.layers.MaxPooling2D(pool_size=(2, 2), padding="valid", strides=2)(x)

x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(1)(x)
outputs = tf.keras.layers.LeakyReLU(alpha=0.3)(x)

model = tf.keras.Model([inputs], outputs)

model.compile("adam", loss="MeanAbsoluteError")

### Train 

In [ ]:
model.fit(
    data.x.train,
    data.y.train,
    epochs=500,
    callbacks=cbs,
    validation_data=(data.x.valid, data.y.valid),
    verbose=1,
)

### save

In [ ]:
model.save(os.path.join(save_dir, exp_name, "w_last.h5"))
joblib.dump(data.x.scaler, os.path.join(save_dir, exp_name, "scaler.save"))

# FIELD NOT NOISY + PERFECT SEG + FIELD PREC + WINDS

In [ ]:
data = Data_train(path_train_nc, path_valid_nc, path_extra_valid_nc)
data.prepare_input(
    "xco2_noiseless", "weighted_plume", "xco2_noiseless_prec", "u_wind", "v_wind"
)
data.prepare_output_inversion()

## EfficientNetB0 - no gen

In [ ]:
exp_name = "xco2noiseless_prec_plume_winds_Eff_nogen"
wandb.init(project=project_name, name=exp_name)

In [ ]:
cbs = get_modelcheckpoint(True, [], os.path.join(save_dir, exp_name, "w_best.h5"))
history = ExtraValidation((data.x.extra_valid, data.y.extra_valid))
cbs.append(WandbMetricsLogger())
cbs.append(history)

In [ ]:
model_builder = rm.Keras_reg_model_builder("EfficientNetB0", data.x.train.shape[1:], 1)
model = model_builder.get_model()
model.compile("adam", loss=tf.keras.losses.MeanAbsoluteError())

### Train 

In [ ]:
model.fit(
    data.x.train, data.y.train,
    epochs=500,
    callbacks=cbs,
    validation_data=(data.x.valid, data.y.valid),
    verbose=1
)

### save

In [ ]:
model.save(os.path.join(save_dir, exp_name, "w_last.h5"))
joblib.dump(data.x.scaler, os.path.join(save_dir, exp_name, "scaler.save"))

## EfficientNetB0 - with gen

In [ ]:
exp_name = "xco2noiseless_prec_plume_winds_Eff_gen"
wandb.init(project=project_name, name=exp_name)

In [ ]:
cbs = get_modelcheckpoint(True, [], os.path.join(save_dir, exp_name, "w_best.h5"))
history = ExtraValidation((data.x.extra_valid, data.y.extra_valid))
cbs.append(WandbMetricsLogger())
cbs.append(history)

In [ ]:
model_builder = rm.Keras_reg_model_builder("EfficientNetB0", data.x.train.shape[1:], 1)
model = model_builder.get_model()
model.compile("adam", loss=tf.keras.losses.MeanAbsoluteError())

### Train 

In [ ]:
model.fit(
    generator.flow(data.x.train, data.y.train),
    epochs=1000,
    callbacks=cbs,
    validation_data=(data.x.valid, data.y.valid),
    verbose=1
)

### save

In [ ]:
model.save(os.path.join(save_dir, exp_name, "w_last.h5"))
joblib.dump(data.x.scaler, os.path.join(save_dir, exp_name, "scaler.save"))

## Linear baseline

In [ ]:
exp_name = "xco2noiseless_prec_plume_winds__linear"
wandb.init(project=project_name, name=exp_name)

In [ ]:
cbs = get_modelcheckpoint(True, [], os.path.join(save_dir, exp_name, "w_best.h5"))
history = ExtraValidation((data.x.extra_valid, data.y.extra_valid))
cbs.append(WandbMetricsLogger())
cbs.append(history)

In [ ]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Conv2D(16, (3, 3), input_shape=data.x.fields_input_shape))
model.add(tf.keras.layers.MaxPooling2D((2, 2)))
model.add(tf.keras.layers.Conv2D(16, (3, 3)))
model.add(tf.keras.layers.MaxPooling2D((2, 2)))
model.add(tf.keras.layers.Conv2D(16, (3, 3)))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(16))
model.add(tf.keras.layers.Dense(1))

model.compile("adam", loss="MeanAbsoluteError")

### Train 

In [ ]:
model.fit(
    data.x.train, data.y.train,
    epochs=200,
    callbacks=cbs,
    validation_data=(data.x.valid, data.y.valid),
    verbose=1
)

### save

In [ ]:
model.save(os.path.join(save_dir, exp_name, "w_last.h5"))
joblib.dump(data.x.scaler, os.path.join(save_dir, exp_name, "scaler.save"))

# FIELD NOISY + PERFECT SEG + FIELD PREC + WINDS

In [ ]:
data = Data_train(path_train_nc, path_valid_nc, path_extra_valid_nc)
data.prepare_input("xco2_noisy", "weighted_plume", "xco2_noisy_prec", "u_wind", "v_wind")
data.prepare_output_inversion()

## EfficientNetB0 - no gen

In [ ]:
exp_name = "xco2_prec_plume_winds_Eff_nogen"
wandb.init(project=project_name, name=exp_name)

In [ ]:
cbs = get_modelcheckpoint(True, [], os.path.join(save_dir, exp_name, "w_best.h5"))
history = ExtraValidation((data.x.extra_valid, data.y.extra_valid))
cbs.append(WandbMetricsLogger())
cbs.append(history)

In [ ]:
model_builder = rm.Keras_reg_model_builder("EfficientNetB0", data.x.train.shape[1:], 1)
model = model_builder.get_model()
model.compile("adam", loss=tf.keras.losses.MeanAbsoluteError())

### Train 

In [ ]:
model.fit(
    data.x.train, data.y.train,
    epochs=500,
    callbacks=cbs,
    validation_data=(data.x.valid, data.y.valid),
    verbose=1
)

### save

In [ ]:
model.save(os.path.join(save_dir, exp_name, "w_last.h5"))
joblib.dump(data.x.scaler, os.path.join(save_dir, exp_name, "scaler.save"))

## EfficientNetB0 - with gen

In [ ]:
exp_name = "xco2_prec_plume_winds_Eff_gen"
wandb.init(project=project_name, name=exp_name)

In [ ]:
cbs = get_modelcheckpoint(True, [], os.path.join(save_dir, exp_name, "w_best.h5"))
history = ExtraValidation((data.x.extra_valid, data.y.extra_valid))
cbs.append(WandbMetricsLogger())
cbs.append(history)

In [ ]:
model_builder = rm.Keras_reg_model_builder("EfficientNetB0", data.x.train.shape[1:], 1)
model = model_builder.get_model()
model.compile("adam", loss=tf.keras.losses.MeanAbsoluteError())

### Train 

In [ ]:
model.fit(
    generator.flow(data.x.train, data.y.train),
    epochs=1000,
    callbacks=cbs,
    validation_data=(data.x.valid, data.y.valid),
    verbose=1
)

### save

In [ ]:
model.save(os.path.join(save_dir, exp_name, "w_last.h5"))
joblib.dump(data.x.scaler, os.path.join(save_dir, exp_name, "scaler.save"))

## Linear baseline

In [ ]:
exp_name = "xco2_prec_plume_winds_linear"
wandb.init(project=project_name, name=exp_name)

In [ ]:
cbs = get_modelcheckpoint(True, [], os.path.join(save_dir, exp_name, "w_best.h5"))
history = ExtraValidation((data.x.extra_valid, data.y.extra_valid))
cbs.append(WandbMetricsLogger())
cbs.append(history)

In [ ]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Conv2D(16, (3, 3), input_shape=data.x.fields_input_shape))
model.add(tf.keras.layers.MaxPooling2D((2, 2)))
model.add(tf.keras.layers.Conv2D(16, (3, 3)))
model.add(tf.keras.layers.MaxPooling2D((2, 2)))
model.add(tf.keras.layers.Conv2D(16, (3, 3)))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(16))
model.add(tf.keras.layers.Dense(1))

model.compile("adam", loss="MeanAbsoluteError")

### Train 

In [ ]:
model.fit(
    data.x.train, data.y.train,
    epochs=200,
    callbacks=cbs,
    validation_data=(data.x.valid, data.y.valid),
    verbose=1
)

### save

In [ ]:
model.save(os.path.join(save_dir, exp_name, "w_last.h5"))
joblib.dump(data.x.scaler, os.path.join(save_dir, exp_name, "scaler.save"))

## Eval

In [ ]:
path_eval_nc = path_extra_valid_nc
exp_name = "plume_Eff_nogen"

data = Data_eval(path_eval_nc)
data.prepare_input(model_eval.get_scaler(os.path.join(save_dir, exp_name)), "plume")
data.prepare_output_inversion()
model = model_eval.get_inversion_model(
    os.path.join(save_dir, exp_name), name_w="w_best.h5"
)

In [ ]:
metrics = model_eval.get_inv_metrics_model_on_data(model, data)
print("mae:", np.mean(metrics["mae"]))
print("mape:", np.mean(metrics["mape"]))

In [ ]:
mean_metrics = model_eval.get_inv_mean_loss(data)
print("mae:", np.mean(mean_metrics["mae"]))
print("mape:", np.mean(mean_metrics["mape"]))

In [ ]:
model_eval.get_summary_histo_inversion(model, data)

# FIELD NOT NOISY + GUESS SEG

# FIELD NOISY + GUESS SEG

# XCO2

In [6]:
data = Data_train(path_train_nc, path_valid_nc, path_extra_valid_nc)
data.prepare_input("xco2")
data.prepare_output_inversion()

2023-02-20 11:37:45.053709: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-20 11:37:45.621470: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 14490 MB memory:  -> device: 0, name: Quadro RTX 5000, pci bus id: 0000:65:00.0, compute capability: 7.5


data.x.train.shape (12576, 64, 64, 1)


## EfficientNetB0 - no gen

In [7]:
exp_name = "xco2_noisy_Eff_nogen"
wandb.init(project=project_name, name=exp_name)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: joffreydumont (cerea-daml). Use `wandb login --relogin` to force relogin


In [8]:
cbs = get_modelcheckpoint(True, [], os.path.join(save_dir, exp_name, "w_best.h5"))
history = ExtraValidation((data.x.extra_valid, data.y.extra_valid))
cbs.append(WandbMetricsLogger())
cbs.append(history)

In [9]:
model_builder = rm.Keras_reg_model_builder("EfficientNetB0", data.x.train.shape[1:], 1)
model = model_builder.get_model()
model.compile("adam", loss=tf.keras.losses.MeanAbsoluteError())

2023-02-13 09:34:03.116182: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-13 09:34:17.189794: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 25485 MB memory:  -> device: 0, name: NVIDIA A40, pci bus id: 0000:21:00.0, compute capability: 8.6


### Train 

In [10]:
model.fit(
    data.x.train,
    data.y.train,
    epochs=500,
    callbacks=cbs,
    validation_data=(data.x.valid, data.y.valid),
    verbose=1,
)

Epoch 1/500


2023-02-13 09:34:30.273865: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inmodel/efficientnetb0/block2b_drop/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer
2023-02-13 09:34:55.594642: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8401
2023-02-13 09:35:18.539993: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-02-13 09:35:20.929530: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:630] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2023-02-13 09:35:21.743544: I tensorflow/compiler/xla/service/service.cc:173] XLA service 0x7f8ada8988d0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-02-13 09:35:21.743583: I tensorflow/compiler/xla/service/service.cc:181]   Stre

393/393 [==============================] - ETA: 0s - loss: 3.8738
Epoch 1: val_loss improved from inf to 5.33688, saving model to /cerea_raid/users/dumontj/dev/coco2/dl/nb_train/sim/pat/plume_Eff_nogen/w_best.h5
extra_val_loss: 1.4520275592803955
393/393 [==============================] - 97s 44ms/step - loss: 3.8738 - val_loss: 5.3369
Epoch 2/500
393/393 [==============================] - ETA: 0s - loss: 2.8868
Epoch 2: val_loss did not improve from 5.33688
extra_val_loss: 1.8317114114761353
393/393 [==============================] - 14s 34ms/step - loss: 2.8868 - val_loss: 5.4245
Epoch 3/500
392/393 [============================>.] - ETA: 0s - loss: 2.8092
Epoch 3: val_loss did not improve from 5.33688
extra_val_loss: 3.102468490600586
393/393 [==============================] - 15s 38ms/step - loss: 2.8078 - val_loss: 6.2284
Epoch 4/500
393/393 [==============================] - ETA: 0s - loss: 2.5587
Epoch 4: val_loss improved from 5.33688 to 2.45140, saving model to /cerea_raid/use

### save

In [11]:
model.save(os.path.join(save_dir, exp_name, "w_last.h5"))
joblib.dump(data.x.scaler, os.path.join(save_dir, exp_name, "scaler.save"))

['/cerea_raid/users/dumontj/dev/coco2/dl/nb_train/sim/pat/plume_Eff_nogen/scaler.save']

## EfficientNetB0 - with gen

In [12]:
exp_name = "xco2_noisy_Eff_gen"
wandb.init(project=project_name, name=exp_name)

epoch/epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
epoch/learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/loss,█▆▅▄▃▃▃▂▂▂▂▄▂▂▂▂▂▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/val_loss,▃▂▁▁▁█▂▁▂▁▃▂▁▁▁▁▁▁▁▂▁▂▁▂▂▂▂▂▂▁▁▁▂▁▂▂▂▂▂▂
extra_val_loss,▁▂▁▂▂█▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▂▂▂▂▂▂▂
epoch/epoch,499
epoch/learning_rate,0.001
epoch/loss,0.26639
epoch/val_loss,2.19401
extra_val_loss,3.26104


In [13]:
cbs = get_modelcheckpoint(True, [], os.path.join(save_dir, exp_name, "w_best.h5"))
history = ExtraValidation((data.x.extra_valid, data.y.extra_valid))
cbs.append(WandbMetricsLogger())
cbs.append(history)

In [14]:
model_builder = rm.Keras_reg_model_builder("EfficientNetB0", data.x.train.shape[1:], 1)
model = model_builder.get_model()
model.compile("adam", loss=tf.keras.losses.MeanAbsoluteError())

### Train 

In [15]:
model.fit(
    generator.flow(data.x.train, data.y.train),
    epochs=1000,
    callbacks=cbs,
    validation_data=(data.x.valid, data.y.valid),
    verbose=1,
)

Epoch 1/1000


2023-02-13 11:44:20.891765: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inmodel_1/efficientnetb0/block2b_drop/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


393/393 [==============================] - ETA: 0s - loss: 4.2316
Epoch 1: val_loss improved from inf to 4.91550, saving model to /cerea_raid/users/dumontj/dev/coco2/dl/nb_train/sim/pat/plume_Eff_gen/w_best.h5
extra_val_loss: 2.102231025695801
393/393 [==============================] - 39s 53ms/step - loss: 4.2316 - val_loss: 4.9155
Epoch 2/1000
392/393 [============================>.] - ETA: 0s - loss: 3.1995
Epoch 2: val_loss did not improve from 4.91550
extra_val_loss: 2.1254639625549316
393/393 [==============================] - 20s 51ms/step - loss: 3.1995 - val_loss: 5.4380
Epoch 3/1000
392/393 [============================>.] - ETA: 0s - loss: 3.0561
Epoch 3: val_loss improved from 4.91550 to 4.58871, saving model to /cerea_raid/users/dumontj/dev/coco2/dl/nb_train/sim/pat/plume_Eff_gen/w_best.h5
extra_val_loss: 1.8833814859390259
393/393 [==============================] - 20s 51ms/step - loss: 3.0552 - val_loss: 4.5887
Epoch 4/1000
393/393 [==============================] - ETA:

### save

In [16]:
model.save(os.path.join(save_dir, exp_name, "w_last.h5"))
joblib.dump(data.x.scaler, os.path.join(save_dir, exp_name, "scaler.save"))

['/cerea_raid/users/dumontj/dev/coco2/dl/nb_train/sim/pat/plume_Eff_gen/scaler.save']

## Linear baseline

In [17]:
exp_name = "xco2_noisy_linear"
wandb.init(project=project_name, name=exp_name)

epoch/epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
epoch/learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/loss,█▇▆▅▅▄▄▄▄▃▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁
epoch/val_loss,▆█▆▃█▃▂▁▂▂▁▂▁▁▁▁▂▁▁▁▁▁▁▁▂▁▁▁▂▁▂▁▁▁▁▁▁▁▁▁
extra_val_loss,▁▁▂▃█▃▄▄▄▃▃▃▃▃▄▃▃▃▃▄▄▄▃▄▃▃▄▃▃▄▃▄▄▃▃▄▄▃▄▄
epoch/epoch,999
epoch/learning_rate,0.001
epoch/loss,1.54357
epoch/val_loss,1.94899
extra_val_loss,4.55071


In [18]:
cbs = get_modelcheckpoint(True, [], os.path.join(save_dir, exp_name, "w_best.h5"))
history = ExtraValidation((data.x.extra_valid, data.y.extra_valid))
cbs.append(WandbMetricsLogger())
cbs.append(history)

In [19]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Conv2D(16, (3, 3), input_shape=data.x.fields_input_shape))
model.add(tf.keras.layers.MaxPooling2D((2, 2)))
model.add(tf.keras.layers.Conv2D(16, (3, 3)))
model.add(tf.keras.layers.MaxPooling2D((2, 2)))
model.add(tf.keras.layers.Conv2D(16, (3, 3)))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(16))
model.add(tf.keras.layers.Dense(1))

model.compile("adam", loss="MeanAbsoluteError")

### Train 

In [20]:
model.fit(
    data.x.train,
    data.y.train,
    epochs=200,
    callbacks=cbs,
    validation_data=(data.x.valid, data.y.valid),
    verbose=1,
)

Epoch 1/200
384/393 [============================>.] - ETA: 0s - loss: 3.7989
Epoch 1: val_loss improved from inf to 3.48380, saving model to /cerea_raid/users/dumontj/dev/coco2/dl/nb_train/sim/pat/plume_linear/w_best.h5
extra_val_loss: 4.0431342124938965
393/393 [==============================] - 3s 5ms/step - loss: 3.7961 - val_loss: 3.4838
Epoch 2/200
382/393 [============================>.] - ETA: 0s - loss: 3.3576
Epoch 2: val_loss did not improve from 3.48380
extra_val_loss: 5.233192443847656
393/393 [==============================] - 2s 4ms/step - loss: 3.3493 - val_loss: 3.5015
Epoch 3/200
379/393 [===========================>..] - ETA: 0s - loss: 3.2980
Epoch 3: val_loss improved from 3.48380 to 3.35223, saving model to /cerea_raid/users/dumontj/dev/coco2/dl/nb_train/sim/pat/plume_linear/w_best.h5
extra_val_loss: 5.039037227630615
393/393 [==============================] - 2s 4ms/step - loss: 3.2964 - val_loss: 3.3522
Epoch 4/200
380/393 [============================>.] - ETA:

### save

In [21]:
model.save(os.path.join(save_dir, exp_name, "w_last.h5"))
joblib.dump(data.x.scaler, os.path.join(save_dir, exp_name, "scaler.save"))

['/cerea_raid/users/dumontj/dev/coco2/dl/nb_train/sim/pat/plume_linear/scaler.save']

## Research of a new model

In [12]:
exp_name = "xco2_noisy_modelX"
wandb.init(project=project_name, name=exp_name)

epoch/epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
epoch/learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/loss,█▆▅▄▄▃▃▃▂▂▃▂▂▂▂▂▂▂▂▂▂▁▂▁▁▁▁▁▁▁▅▂▁▁▁▁▁▁▁▁
epoch/val_loss,▇█▅▅▄▇▅▇▄▂█▂▅▃▂▃▁▃▂▁▁▃▂▁▁▃▃▃▃▄▃▂▂▂▃▁▃▂▂▁
extra_val_loss,▅▃▇▃▅█▇▂▄▄▂▅▅▄▃▆▄▄▂▃▃▃▃▅▅▃▂▂▄▅▇▅▆▃▃▄▁▃▅▄
epoch/epoch,499
epoch/learning_rate,0.001
epoch/loss,3.14929
epoch/val_loss,3.59156
extra_val_loss,2.49425


Problem at: /tmp/ipykernel_1410243/4155257584.py 2 <module>


KeyboardInterrupt: 

Error in callback <bound method AutoreloadMagics.post_execute_hook of <IPython.extensions.autoreload.AutoreloadMagics object at 0x7f7e4fdee7c0>> (for post_execute):


KeyboardInterrupt: 

In [8]:
cbs = get_modelcheckpoint(True, [], os.path.join(save_dir, exp_name, "w_best.h5"))
history = ExtraValidation((data.x.extra_valid, data.y.extra_valid))
cbs.append(WandbMetricsLogger())
cbs.append(history)

In [9]:
reg_builder = rm.Reg_model_builder(
    "essential",
    data.x.fields_input_shape,
    data.y.classes,
    data.x.n_layer,
    data.x.xco2_noisy_chans,
)
model = reg_builder.get_model()
model.compile("adam", "MeanAbsoluteError")

### Train 

In [10]:
model.fit(
    data.x.train,
    data.y.train,
    epochs=1000,
    callbacks=cbs,
    validation_data=(data.x.valid, data.y.valid),
    verbose=1,
)

Epoch 1/500


2023-02-20 11:38:01.953264: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inmodel_1/model/dropout/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer
2023-02-20 11:38:02.699351: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8401
2023-02-20 11:38:03.261296: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-02-20 11:38:03.609277: I tensorflow/compiler/xla/service/service.cc:173] XLA service 0x7f77996df360 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-02-20 11:38:03.609305: I tensorflow/compiler/xla/service/service.cc:181]   StreamExecutor device (0): Quadro RTX 5000, Compute Capability 7.5
2023-02-20 11:38:03.612774: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set 

391/393 [============================>.] - ETA: 0s - loss: 4.4488
Epoch 1: val_loss improved from inf to 4.30865, saving model to /cerea_raid/users/dumontj/dev/coco2/dl/nb_train/sim/pat/plume_modelX/w_best.h5
extra_val_loss: 3.0225837230682373
393/393 [==============================] - 10s 13ms/step - loss: 4.4493 - val_loss: 4.3087
Epoch 2/500
391/393 [============================>.] - ETA: 0s - loss: 4.2343
Epoch 2: val_loss did not improve from 4.30865
extra_val_loss: 2.441567897796631
393/393 [==============================] - 5s 12ms/step - loss: 4.2364 - val_loss: 4.3639
Epoch 3/500
391/393 [============================>.] - ETA: 0s - loss: 4.1681
Epoch 3: val_loss improved from 4.30865 to 4.17098, saving model to /cerea_raid/users/dumontj/dev/coco2/dl/nb_train/sim/pat/plume_modelX/w_best.h5
extra_val_loss: 3.3424787521362305
393/393 [==============================] - 5s 13ms/step - loss: 4.1620 - val_loss: 4.1710
Epoch 4/500
391/393 [============================>.] - ETA: 0s - l

### save

In [13]:
model.save(os.path.join(save_dir, exp_name, "w_last.h5"))

# XCO2 - XCO2_prec - winds - perf bool seg - perf seg

In [8]:
data = Data_train(path_train_nc, path_valid_nc, path_extra_valid_nc)
data.prepare_input("xco2", "xco2_prec", "u_wind", "v_wind", "bool_perf_seg")
data.prepare_output_inversion()

2023-02-24 10:50:43.375852: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 2833121280 exceeds 10% of free system memory.
2023-02-24 10:50:45.211191: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 2833121280 exceeds 10% of free system memory.


data.x.train.shape (34584, 64, 64, 5)


## EfficientNetB0 - no gen

In [7]:
exp_name = "xco2_prec_winds_noisy_Eff_nogen"
wandb.init(project=project_name, name=exp_name)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: joffreydumont (cerea-daml). Use `wandb login --relogin` to force relogin


In [8]:
cbs = get_modelcheckpoint(True, [], os.path.join(save_dir, exp_name, "w_best.h5"))
history = ExtraValidation((data.x.extra_valid, data.y.extra_valid))
cbs.append(WandbMetricsLogger())
cbs.append(history)

In [9]:
model_builder = rm.Keras_reg_model_builder("EfficientNetB0", data.x.train.shape[1:], 1)
model = model_builder.get_model()
model.compile("adam", loss=tf.keras.losses.MeanAbsoluteError())

2023-02-13 09:34:03.116182: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-13 09:34:17.189794: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 25485 MB memory:  -> device: 0, name: NVIDIA A40, pci bus id: 0000:21:00.0, compute capability: 8.6


### Train 

In [10]:
model.fit(
    data.x.train,
    data.y.train,
    epochs=500,
    callbacks=cbs,
    validation_data=(data.x.valid, data.y.valid),
    verbose=1,
)

Epoch 1/500


2023-02-13 09:34:30.273865: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inmodel/efficientnetb0/block2b_drop/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer
2023-02-13 09:34:55.594642: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8401
2023-02-13 09:35:18.539993: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-02-13 09:35:20.929530: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:630] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2023-02-13 09:35:21.743544: I tensorflow/compiler/xla/service/service.cc:173] XLA service 0x7f8ada8988d0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-02-13 09:35:21.743583: I tensorflow/compiler/xla/service/service.cc:181]   Stre

393/393 [==============================] - ETA: 0s - loss: 3.8738
Epoch 1: val_loss improved from inf to 5.33688, saving model to /cerea_raid/users/dumontj/dev/coco2/dl/nb_train/sim/pat/plume_Eff_nogen/w_best.h5
extra_val_loss: 1.4520275592803955
393/393 [==============================] - 97s 44ms/step - loss: 3.8738 - val_loss: 5.3369
Epoch 2/500
393/393 [==============================] - ETA: 0s - loss: 2.8868
Epoch 2: val_loss did not improve from 5.33688
extra_val_loss: 1.8317114114761353
393/393 [==============================] - 14s 34ms/step - loss: 2.8868 - val_loss: 5.4245
Epoch 3/500
392/393 [============================>.] - ETA: 0s - loss: 2.8092
Epoch 3: val_loss did not improve from 5.33688
extra_val_loss: 3.102468490600586
393/393 [==============================] - 15s 38ms/step - loss: 2.8078 - val_loss: 6.2284
Epoch 4/500
393/393 [==============================] - ETA: 0s - loss: 2.5587
Epoch 4: val_loss improved from 5.33688 to 2.45140, saving model to /cerea_raid/use

### save

In [11]:
model.save(os.path.join(save_dir, exp_name, "w_last.h5"))
joblib.dump(data.x.scaler, os.path.join(save_dir, exp_name, "scaler.save"))

['/cerea_raid/users/dumontj/dev/coco2/dl/nb_train/sim/pat/plume_Eff_nogen/scaler.save']

## EfficientNetB0 - with gen

In [12]:
exp_name = "xco2_prec_winds_noisy_Eff_gen"
wandb.init(project=project_name, name=exp_name)

epoch/epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
epoch/learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/loss,█▆▅▄▃▃▃▂▂▂▂▄▂▂▂▂▂▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/val_loss,▃▂▁▁▁█▂▁▂▁▃▂▁▁▁▁▁▁▁▂▁▂▁▂▂▂▂▂▂▁▁▁▂▁▂▂▂▂▂▂
extra_val_loss,▁▂▁▂▂█▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▂▂▂▂▂▂▂
epoch/epoch,499
epoch/learning_rate,0.001
epoch/loss,0.26639
epoch/val_loss,2.19401
extra_val_loss,3.26104


In [13]:
cbs = get_modelcheckpoint(True, [], os.path.join(save_dir, exp_name, "w_best.h5"))
history = ExtraValidation((data.x.extra_valid, data.y.extra_valid))
cbs.append(WandbMetricsLogger())
cbs.append(history)

In [14]:
model_builder = rm.Keras_reg_model_builder("EfficientNetB0", data.x.train.shape[1:], 1)
model = model_builder.get_model()
model.compile("adam", loss=tf.keras.losses.MeanAbsoluteError())

### Train 

In [15]:
model.fit(
    generator.flow(data.x.train, data.y.train),
    epochs=1000,
    callbacks=cbs,
    validation_data=(data.x.valid, data.y.valid),
    verbose=1,
)

Epoch 1/1000


2023-02-13 11:44:20.891765: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inmodel_1/efficientnetb0/block2b_drop/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


393/393 [==============================] - ETA: 0s - loss: 4.2316
Epoch 1: val_loss improved from inf to 4.91550, saving model to /cerea_raid/users/dumontj/dev/coco2/dl/nb_train/sim/pat/plume_Eff_gen/w_best.h5
extra_val_loss: 2.102231025695801
393/393 [==============================] - 39s 53ms/step - loss: 4.2316 - val_loss: 4.9155
Epoch 2/1000
392/393 [============================>.] - ETA: 0s - loss: 3.1995
Epoch 2: val_loss did not improve from 4.91550
extra_val_loss: 2.1254639625549316
393/393 [==============================] - 20s 51ms/step - loss: 3.1995 - val_loss: 5.4380
Epoch 3/1000
392/393 [============================>.] - ETA: 0s - loss: 3.0561
Epoch 3: val_loss improved from 4.91550 to 4.58871, saving model to /cerea_raid/users/dumontj/dev/coco2/dl/nb_train/sim/pat/plume_Eff_gen/w_best.h5
extra_val_loss: 1.8833814859390259
393/393 [==============================] - 20s 51ms/step - loss: 3.0552 - val_loss: 4.5887
Epoch 4/1000
393/393 [==============================] - ETA:

### save

In [16]:
model.save(os.path.join(save_dir, exp_name, "w_last.h5"))
joblib.dump(data.x.scaler, os.path.join(save_dir, exp_name, "scaler.save"))

['/cerea_raid/users/dumontj/dev/coco2/dl/nb_train/sim/pat/plume_Eff_gen/scaler.save']

## Linear baseline

In [17]:
exp_name = "xco2_prec_winds_noisy_linear"
wandb.init(project=project_name, name=exp_name)

epoch/epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
epoch/learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/loss,█▇▆▅▅▄▄▄▄▃▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁
epoch/val_loss,▆█▆▃█▃▂▁▂▂▁▂▁▁▁▁▂▁▁▁▁▁▁▁▂▁▁▁▂▁▂▁▁▁▁▁▁▁▁▁
extra_val_loss,▁▁▂▃█▃▄▄▄▃▃▃▃▃▄▃▃▃▃▄▄▄▃▄▃▃▄▃▃▄▃▄▄▃▃▄▄▃▄▄
epoch/epoch,999
epoch/learning_rate,0.001
epoch/loss,1.54357
epoch/val_loss,1.94899
extra_val_loss,4.55071


In [18]:
cbs = get_modelcheckpoint(True, [], os.path.join(save_dir, exp_name, "w_best.h5"))
history = ExtraValidation((data.x.extra_valid, data.y.extra_valid))
cbs.append(WandbMetricsLogger())
cbs.append(history)

In [19]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Conv2D(16, (3, 3), input_shape=data.x.fields_input_shape))
model.add(tf.keras.layers.MaxPooling2D((2, 2)))
model.add(tf.keras.layers.Conv2D(16, (3, 3)))
model.add(tf.keras.layers.MaxPooling2D((2, 2)))
model.add(tf.keras.layers.Conv2D(16, (3, 3)))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(16))
model.add(tf.keras.layers.Dense(1))

model.compile("adam", loss="MeanAbsoluteError")

### Train 

In [20]:
model.fit(
    data.x.train,
    data.y.train,
    epochs=200,
    callbacks=cbs,
    validation_data=(data.x.valid, data.y.valid),
    verbose=1,
)

Epoch 1/200
384/393 [============================>.] - ETA: 0s - loss: 3.7989
Epoch 1: val_loss improved from inf to 3.48380, saving model to /cerea_raid/users/dumontj/dev/coco2/dl/nb_train/sim/pat/plume_linear/w_best.h5
extra_val_loss: 4.0431342124938965
393/393 [==============================] - 3s 5ms/step - loss: 3.7961 - val_loss: 3.4838
Epoch 2/200
382/393 [============================>.] - ETA: 0s - loss: 3.3576
Epoch 2: val_loss did not improve from 3.48380
extra_val_loss: 5.233192443847656
393/393 [==============================] - 2s 4ms/step - loss: 3.3493 - val_loss: 3.5015
Epoch 3/200
379/393 [===========================>..] - ETA: 0s - loss: 3.2980
Epoch 3: val_loss improved from 3.48380 to 3.35223, saving model to /cerea_raid/users/dumontj/dev/coco2/dl/nb_train/sim/pat/plume_linear/w_best.h5
extra_val_loss: 5.039037227630615
393/393 [==============================] - 2s 4ms/step - loss: 3.2964 - val_loss: 3.3522
Epoch 4/200
380/393 [============================>.] - ETA:

### save

In [21]:
model.save(os.path.join(save_dir, exp_name, "w_last.h5"))
joblib.dump(data.x.scaler, os.path.join(save_dir, exp_name, "scaler.save"))

['/cerea_raid/users/dumontj/dev/coco2/dl/nb_train/sim/pat/plume_linear/scaler.save']

## Research of a new model

In [9]:
exp_name = "xco2_prec_winds_noisy_perfseg_boolperfseg_modelX"
wandb.init(project=project_name, name=exp_name)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: joffreydumont (cerea-daml). Use `wandb login --relogin` to force relogin


In [10]:
cbs = get_modelcheckpoint(True, [], os.path.join(save_dir, exp_name, "w_best.h5"))
history = ExtraValidation((data.x.extra_valid, data.y.extra_valid))
cbs.append(WandbMetricsLogger())
cbs.append(history)

In [11]:
reg_builder = rm.Reg_model_builder(
    "essential",
    data.x.fields_input_shape,
    data.y.classes,
    data.x.n_layer,
    data.x.xco2_noisy_chans,
)
model = reg_builder.get_model()
model.compile("adam", "MeanAbsoluteError")

### Train 

In [ ]:
model.fit(
    data.x.train,
    data.y.train,
    epochs=2000,
    callbacks=cbs,
    validation_data=(data.x.valid, data.y.valid),
    verbose=1,
)

2023-02-24 10:51:09.249296: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 2833121280 exceeds 10% of free system memory.


Epoch 1/2000


2023-02-24 10:51:13.369266: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inmodel_1/model/dropout/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer
2023-02-24 10:51:15.567522: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8401
2023-02-24 10:51:17.468987: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-02-24 10:51:17.877043: I tensorflow/compiler/xla/service/service.cc:173] XLA service 0x7fa758017ff0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-02-24 10:51:17.877097: I tensorflow/compiler/xla/service/service.cc:181]   StreamExecutor device (0): Quadro RTX 5000, Compute Capability 7.5
2023-02-24 10:51:17.887316: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set 

1081/1081 [==============================] - ETA: 0s - loss: 7.0730
Epoch 1: val_loss improved from inf to 5.81417, saving model to /cerea_raid/users/dumontj/dev/coco2/dl/nb_train/sim/lip/xco2_prec_winds_noisy_perfseg_boolperfseg_modelX/w_best.h5
extra_val_loss: 4.359269142150879
1081/1081 [==============================] - 26s 17ms/step - loss: 7.0730 - val_loss: 5.8142
Epoch 2/2000
1081/1081 [==============================] - ETA: 0s - loss: 4.9598
Epoch 2: val_loss improved from 5.81417 to 4.94662, saving model to /cerea_raid/users/dumontj/dev/coco2/dl/nb_train/sim/lip/xco2_prec_winds_noisy_perfseg_boolperfseg_modelX/w_best.h5
extra_val_loss: 3.9618825912475586
1081/1081 [==============================] - 17s 16ms/step - loss: 4.9598 - val_loss: 4.9466
Epoch 3/2000
1079/1081 [============================>.] - ETA: 0s - loss: 4.3020
Epoch 3: val_loss improved from 4.94662 to 4.68222, saving model to /cerea_raid/users/dumontj/dev/coco2/dl/nb_train/sim/lip/xco2_prec_winds_noisy_perfseg

### save

In [13]:
model.save(os.path.join(save_dir, exp_name, "w_last.h5"))

# XCO2 - XCO2_prec - winds

In [15]:
data = Data_train(path_train_nc, path_valid_nc, path_extra_valid_nc)
data.prepare_input("xco2", "xco2_prec", "u_wind", "v_wind")
data.prepare_output_inversion()

2023-02-23 16:52:32.382549: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 2266497024 exceeds 10% of free system memory.
2023-02-23 16:52:33.838545: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 2266497024 exceeds 10% of free system memory.


data.x.train.shape (34584, 64, 64, 4)


## EfficientNetB0 - no gen

In [7]:
exp_name = "xco2_prec_winds_noisy_Eff_nogen"
wandb.init(project=project_name, name=exp_name)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: joffreydumont (cerea-daml). Use `wandb login --relogin` to force relogin


In [8]:
cbs = get_modelcheckpoint(True, [], os.path.join(save_dir, exp_name, "w_best.h5"))
history = ExtraValidation((data.x.extra_valid, data.y.extra_valid))
cbs.append(WandbMetricsLogger())
cbs.append(history)

In [9]:
model_builder = rm.Keras_reg_model_builder("EfficientNetB0", data.x.train.shape[1:], 1)
model = model_builder.get_model()
model.compile("adam", loss=tf.keras.losses.MeanAbsoluteError())

2023-02-13 09:34:03.116182: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-13 09:34:17.189794: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 25485 MB memory:  -> device: 0, name: NVIDIA A40, pci bus id: 0000:21:00.0, compute capability: 8.6


### Train 

In [10]:
model.fit(
    data.x.train,
    data.y.train,
    epochs=500,
    callbacks=cbs,
    validation_data=(data.x.valid, data.y.valid),
    verbose=1,
)

Epoch 1/500


2023-02-13 09:34:30.273865: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inmodel/efficientnetb0/block2b_drop/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer
2023-02-13 09:34:55.594642: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8401
2023-02-13 09:35:18.539993: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-02-13 09:35:20.929530: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:630] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2023-02-13 09:35:21.743544: I tensorflow/compiler/xla/service/service.cc:173] XLA service 0x7f8ada8988d0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-02-13 09:35:21.743583: I tensorflow/compiler/xla/service/service.cc:181]   Stre

393/393 [==============================] - ETA: 0s - loss: 3.8738
Epoch 1: val_loss improved from inf to 5.33688, saving model to /cerea_raid/users/dumontj/dev/coco2/dl/nb_train/sim/pat/plume_Eff_nogen/w_best.h5
extra_val_loss: 1.4520275592803955
393/393 [==============================] - 97s 44ms/step - loss: 3.8738 - val_loss: 5.3369
Epoch 2/500
393/393 [==============================] - ETA: 0s - loss: 2.8868
Epoch 2: val_loss did not improve from 5.33688
extra_val_loss: 1.8317114114761353
393/393 [==============================] - 14s 34ms/step - loss: 2.8868 - val_loss: 5.4245
Epoch 3/500
392/393 [============================>.] - ETA: 0s - loss: 2.8092
Epoch 3: val_loss did not improve from 5.33688
extra_val_loss: 3.102468490600586
393/393 [==============================] - 15s 38ms/step - loss: 2.8078 - val_loss: 6.2284
Epoch 4/500
393/393 [==============================] - ETA: 0s - loss: 2.5587
Epoch 4: val_loss improved from 5.33688 to 2.45140, saving model to /cerea_raid/use

### save

In [11]:
model.save(os.path.join(save_dir, exp_name, "w_last.h5"))
joblib.dump(data.x.scaler, os.path.join(save_dir, exp_name, "scaler.save"))

['/cerea_raid/users/dumontj/dev/coco2/dl/nb_train/sim/pat/plume_Eff_nogen/scaler.save']

## EfficientNetB0 - with gen

In [12]:
exp_name = "xco2_prec_winds_noisy_Eff_gen"
wandb.init(project=project_name, name=exp_name)

epoch/epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
epoch/learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/loss,█▆▅▄▃▃▃▂▂▂▂▄▂▂▂▂▂▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/val_loss,▃▂▁▁▁█▂▁▂▁▃▂▁▁▁▁▁▁▁▂▁▂▁▂▂▂▂▂▂▁▁▁▂▁▂▂▂▂▂▂
extra_val_loss,▁▂▁▂▂█▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▂▂▂▂▂▂▂
epoch/epoch,499
epoch/learning_rate,0.001
epoch/loss,0.26639
epoch/val_loss,2.19401
extra_val_loss,3.26104


In [13]:
cbs = get_modelcheckpoint(True, [], os.path.join(save_dir, exp_name, "w_best.h5"))
history = ExtraValidation((data.x.extra_valid, data.y.extra_valid))
cbs.append(WandbMetricsLogger())
cbs.append(history)

In [14]:
model_builder = rm.Keras_reg_model_builder("EfficientNetB0", data.x.train.shape[1:], 1)
model = model_builder.get_model()
model.compile("adam", loss=tf.keras.losses.MeanAbsoluteError())

### Train 

In [15]:
model.fit(
    generator.flow(data.x.train, data.y.train),
    epochs=1000,
    callbacks=cbs,
    validation_data=(data.x.valid, data.y.valid),
    verbose=1,
)

Epoch 1/1000


2023-02-13 11:44:20.891765: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inmodel_1/efficientnetb0/block2b_drop/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


393/393 [==============================] - ETA: 0s - loss: 4.2316
Epoch 1: val_loss improved from inf to 4.91550, saving model to /cerea_raid/users/dumontj/dev/coco2/dl/nb_train/sim/pat/plume_Eff_gen/w_best.h5
extra_val_loss: 2.102231025695801
393/393 [==============================] - 39s 53ms/step - loss: 4.2316 - val_loss: 4.9155
Epoch 2/1000
392/393 [============================>.] - ETA: 0s - loss: 3.1995
Epoch 2: val_loss did not improve from 4.91550
extra_val_loss: 2.1254639625549316
393/393 [==============================] - 20s 51ms/step - loss: 3.1995 - val_loss: 5.4380
Epoch 3/1000
392/393 [============================>.] - ETA: 0s - loss: 3.0561
Epoch 3: val_loss improved from 4.91550 to 4.58871, saving model to /cerea_raid/users/dumontj/dev/coco2/dl/nb_train/sim/pat/plume_Eff_gen/w_best.h5
extra_val_loss: 1.8833814859390259
393/393 [==============================] - 20s 51ms/step - loss: 3.0552 - val_loss: 4.5887
Epoch 4/1000
393/393 [==============================] - ETA:

### save

In [16]:
model.save(os.path.join(save_dir, exp_name, "w_last.h5"))
joblib.dump(data.x.scaler, os.path.join(save_dir, exp_name, "scaler.save"))

['/cerea_raid/users/dumontj/dev/coco2/dl/nb_train/sim/pat/plume_Eff_gen/scaler.save']

## Linear baseline

In [17]:
exp_name = "xco2_prec_winds_noisy_linear"
wandb.init(project=project_name, name=exp_name)

epoch/epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
epoch/learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/loss,█▇▆▅▅▄▄▄▄▃▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁
epoch/val_loss,▆█▆▃█▃▂▁▂▂▁▂▁▁▁▁▂▁▁▁▁▁▁▁▂▁▁▁▂▁▂▁▁▁▁▁▁▁▁▁
extra_val_loss,▁▁▂▃█▃▄▄▄▃▃▃▃▃▄▃▃▃▃▄▄▄▃▄▃▃▄▃▃▄▃▄▄▃▃▄▄▃▄▄
epoch/epoch,999
epoch/learning_rate,0.001
epoch/loss,1.54357
epoch/val_loss,1.94899
extra_val_loss,4.55071


In [18]:
cbs = get_modelcheckpoint(True, [], os.path.join(save_dir, exp_name, "w_best.h5"))
history = ExtraValidation((data.x.extra_valid, data.y.extra_valid))
cbs.append(WandbMetricsLogger())
cbs.append(history)

In [19]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Conv2D(16, (3, 3), input_shape=data.x.fields_input_shape))
model.add(tf.keras.layers.MaxPooling2D((2, 2)))
model.add(tf.keras.layers.Conv2D(16, (3, 3)))
model.add(tf.keras.layers.MaxPooling2D((2, 2)))
model.add(tf.keras.layers.Conv2D(16, (3, 3)))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(16))
model.add(tf.keras.layers.Dense(1))

model.compile("adam", loss="MeanAbsoluteError")

### Train 

In [20]:
model.fit(
    data.x.train,
    data.y.train,
    epochs=200,
    callbacks=cbs,
    validation_data=(data.x.valid, data.y.valid),
    verbose=1,
)

Epoch 1/200
384/393 [============================>.] - ETA: 0s - loss: 3.7989
Epoch 1: val_loss improved from inf to 3.48380, saving model to /cerea_raid/users/dumontj/dev/coco2/dl/nb_train/sim/pat/plume_linear/w_best.h5
extra_val_loss: 4.0431342124938965
393/393 [==============================] - 3s 5ms/step - loss: 3.7961 - val_loss: 3.4838
Epoch 2/200
382/393 [============================>.] - ETA: 0s - loss: 3.3576
Epoch 2: val_loss did not improve from 3.48380
extra_val_loss: 5.233192443847656
393/393 [==============================] - 2s 4ms/step - loss: 3.3493 - val_loss: 3.5015
Epoch 3/200
379/393 [===========================>..] - ETA: 0s - loss: 3.2980
Epoch 3: val_loss improved from 3.48380 to 3.35223, saving model to /cerea_raid/users/dumontj/dev/coco2/dl/nb_train/sim/pat/plume_linear/w_best.h5
extra_val_loss: 5.039037227630615
393/393 [==============================] - 2s 4ms/step - loss: 3.2964 - val_loss: 3.3522
Epoch 4/200
380/393 [============================>.] - ETA:

### save

In [21]:
model.save(os.path.join(save_dir, exp_name, "w_last.h5"))
joblib.dump(data.x.scaler, os.path.join(save_dir, exp_name, "scaler.save"))

['/cerea_raid/users/dumontj/dev/coco2/dl/nb_train/sim/pat/plume_linear/scaler.save']

## Research of a new model

In [16]:
exp_name = "xco2_prec_winds_noisy_modelX"
wandb.init(project=project_name, name=exp_name)

epoch/epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
epoch/learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/loss,█▇▆▅▅▄▄▄▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/val_loss,█▄▁▂▂▄▄▅▆▃▅▃▄▅▇▆▃▅▄▂▄▂▄▃▄▂▅▄▄▃▅▄▂▄▂▂▂▄▃▃
extra_val_loss,█▃▃▂▄▃▁▅▆▃▅▂▃▁▅▁▂▅▃▂▃▂▅▃▃▃▅▄▃▄▅▂▃▄▃▂▃▄▃▂
epoch/epoch,219
epoch/learning_rate,0.001
epoch/loss,2.92147
epoch/val_loss,4.71965
extra_val_loss,3.19237


In [17]:
cbs = get_modelcheckpoint(True, [], os.path.join(save_dir, exp_name, "w_best.h5"))
history = ExtraValidation((data.x.extra_valid, data.y.extra_valid))
cbs.append(WandbMetricsLogger())
cbs.append(history)

In [18]:
reg_builder = rm.Reg_model_builder(
    "essential",
    data.x.fields_input_shape,
    data.y.classes,
    data.x.n_layer,
    data.x.xco2_noisy_chans,
)
model = reg_builder.get_model()
model.compile("adam", "MeanAbsoluteError")

### Train 

In [ ]:
model.fit(
    data.x.train,
    data.y.train,
    epochs=2000,
    callbacks=cbs,
    validation_data=(data.x.valid, data.y.valid),
    verbose=1,
)

2023-02-23 16:52:54.885411: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 2266497024 exceeds 10% of free system memory.
2023-02-23 16:52:56.278927: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 2266497024 exceeds 10% of free system memory.


Epoch 1/2000


2023-02-23 16:52:58.328228: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inmodel_2/model_1/dropout_3/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


1081/1081 [==============================] - ETA: 0s - loss: 5.4028
Epoch 1: val_loss improved from inf to 5.09391, saving model to /cerea_raid/users/dumontj/dev/coco2/dl/nb_train/sim/lip/xco2_prec_winds_noisy_modelX/w_best.h5
extra_val_loss: 4.028751850128174
1081/1081 [==============================] - 21s 17ms/step - loss: 5.4028 - val_loss: 5.0939
Epoch 2/2000
1081/1081 [==============================] - ETA: 0s - loss: 4.3446
Epoch 2: val_loss improved from 5.09391 to 5.06222, saving model to /cerea_raid/users/dumontj/dev/coco2/dl/nb_train/sim/lip/xco2_prec_winds_noisy_modelX/w_best.h5
extra_val_loss: 4.823084831237793
1081/1081 [==============================] - 17s 16ms/step - loss: 4.3446 - val_loss: 5.0622
Epoch 3/2000
1081/1081 [==============================] - ETA: 0s - loss: 4.1198
Epoch 3: val_loss improved from 5.06222 to 4.62907, saving model to /cerea_raid/users/dumontj/dev/coco2/dl/nb_train/sim/lip/xco2_prec_winds_noisy_modelX/w_best.h5
extra_val_loss: 4.2050137519836

### save

In [13]:
model.save(os.path.join(save_dir, exp_name, "w_last.h5"))